# py_函数

## 1 `__doc__`

In [1]:
def factorial(n):
    ''' 计算阶乘 '''
    return 1 if n < 2 else n * factorial(n-1)

In [2]:
print(factorial.__doc__)
print(help(factorial))

 计算阶乘 
Help on function factorial in module __main__:

factorial(n)
    计算阶乘

None


## 2 高阶函数 `map` / `filter` / `reduce`

In [3]:
# 案例: 计算阶乘
print(list(map(factorial, range(6))))
print([factorial(n) for n in range(6)])

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]


In [4]:
from functools import reduce
from operator import add

print(reduce(add, range(100)))
print(sum(range(100)))

4950
4950


### 归约函数 `all(iterable)` / `any(iterable)`

## 3 匿名函数 `lambda`

In [5]:
fruits = ['banana', 'apple', 'origine', 'cherry']
print(sorted(fruits, key=lambda word:word[::-1]))
print(sorted(fruits, key=lambda word:word[0]))
print(sorted(fruits, key=lambda word:len(word)))

['banana', 'apple', 'origine', 'cherry']
['apple', 'banana', 'cherry', 'origine']
['apple', 'banana', 'cherry', 'origine']


## 4 可调用对象
调用运算符 `()`

1. 用户自定义函数
2. 内置函数：用C（Cpython）实现的函数
3. 内置方法：用C实现的方法
4. 方法
5. 类
6. 类的实例
7. 生成器函数

In [6]:
# 判断对象是否可调用
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

## 5 用户定义的可调用类型

In [7]:
# 自定义一个从打乱列表取出一个元素的自定义可调用类型
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick empty BingoCage')
    
    def __call__(self):
        return self.pick()

In [8]:
bingo =BingoCage(range(3))
print(callable(bingo))
for i in range(3):
    print(bingo.pick())

True
0
2
1


## 6 函数内省

In [9]:
dir(bingo)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_items',
 'pick']

In [10]:
class C:
    pass
obj = C()

def func():
    pass

sorted(set(dir(func)) - set(dir(obj)))

# '__annotations__', dict, 参数和返回值的注解
# '__call__', method-wrapper, 实现（）运算符，可调用对象协议
# '__closure__', turple, 函数的闭包，即自由变量的绑定（通常None）
# '__code__', code, 编译成字节码的函数元数据和函数自定义体
# '__defaults__', tuple, 形式参数默认值
# '__get__', method-wrapper, 实现只读描述符协议
# '__globals__', dict, 函数所在模块中的全局变量
# '__kwdefaults__', dict, 函数所在模块中的全局变量
# '__name__', str, 函数名称
# '__qualname__'， str, 函数的限定名称，如Random.choice

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

## 7 从定位参数到仅限关键字参数

In [11]:
def tag(name, *content, cls=None, **attrs):
    """ 生成1个或者多个HTML标签 """
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ' '.join(f"{attr}='{value}'" for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join(f"<{name} {attr_str}>{c}</{name}>" for c in content)
    else:
        return f"<{name} {attr_str}/>"

In [12]:
print(tag('br'))
print(tag('p','hello'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))
print(tag('p', 'world', cls='sidebar', id=33))

my_tag = {'name': 'img', 'title': 'Sunset', 'src': 'sunset.jpg', 'cls': 'framed'}
print(tag(**my_tag))

my_tag1 = {'name': 'img', 'title': 'Sunset', 'src': 'sunset.jpg', 'cls': 'framed', 'content': 'sunset'}
print(tag(**my_tag1))

<br />
<p >hello</p>
<p >hello</p>
<p >world</p>
<p id='33'>hello</p>
<p class='sidebar' id='33'>world</p>
<img class='framed' src='sunset.jpg' title='Sunset'/>
<img class='framed' content='sunset' src='sunset.jpg' title='Sunset'/>


In [13]:
def f(a, *, b):
    return a,b

f(1, b=2)

(1, 2)

## 8 获取关于参数的信息

In [14]:
def clip(text, max_len=80):
    """
    在 max_len 前面或者后面的第一个空格处截断文本。
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ',0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstip()

In [15]:
print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames)  # 参数名称，包含函数内部的局部变量
print(clip.__code__.co_argcount)  # 参数数量，不包含前缀为 * 和 ** 的变长参数 

(80,)
<code object clip at 0x00000234E34CFAE0, file "<ipython-input-14-af8ad0808fa8>", line 1>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


In [16]:
# 提取函数名
# 属性：name, kind, default, inspect.Parameter(annotation)
from inspect import signature

sig = signature(clip)
print(sig)

for name, param in sig.parameters.items():
    print(param.kind, '：', name, '=', param.default)
    
# kind 的属性：
# 1. POSITIONAL_OR_KEYWORD, 可以通过定位参数和关键字参数传入的形参
# 2. VAR_POSITION， 定位参数元祖
# 3. VAR_KEYWORD， 关键字参数字典
# 4. KEYWORD_ONLY，仅限关键字参数
# 5. POSITION_ONLY，仅限定位参数

(text, max_len=80)
POSITIONAL_OR_KEYWORD ： text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD ： max_len = 80


In [17]:
import inspect

sig = inspect.signature(tag)
print(sig)

my_tag = {'name': 'img', 'title': 'sunset boulevard', 'src': 'sunset.jpg', 'cls':'framed'}
# inspect.Signature 对象有一个 bind 方法，可以把任意参数绑定到签名中的形参上，所用的规则与实参到形参的匹配方式一样。
# 框架可以使用这个方法在真正调用函数前验证参数。
bound_args = sig.bind(**my_tag)
print(bound_args)
for name, value in bound_args.arguments.items():
    print(name, '=', value)
    
del my_tag['name']
try:
    bound_args = sig.bind(**my_tag)
except Exception as e:
    print(e)

(name, *content, cls=None, **attrs)
<BoundArguments (name='img', cls='framed', attrs={'title': 'sunset boulevard', 'src': 'sunset.jpg'})>
name = img
cls = framed
attrs = {'title': 'sunset boulevard', 'src': 'sunset.jpg'}
missing a required argument: 'name'


## 9 函数注解

1. 参数有默认值，注解放在参数明和 = 号之间；
2. 注解返回值：在()和:之间，添加 -> 和一个表达式，表达式可以是任何值，最常用的是类 `str` 和字符串 `'int>0'`;
3. 注解不做任何处理，只是存储在函数的 `__annotations__` 属性

In [18]:
def clip(text: str, max_len: 'int > 0'=80) -> str:
    """
    在 max_len 前面或者后面的第一个空格处截断文本。
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ',0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstip()

In [19]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

In [20]:
from inspect import signature
sig = signature(clip)
print(sig.return_annotation)

for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'>
<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


## 10 支持函数式编程的包

### 10.1 operator模块

1. `mul`:
2. `imatmul`:
3. `matmul`:
2. `itemgetter`: 根据元组的某个字段给元组列表排序；
3. `attrgetter`: 根据名称提取对象的属性；
4. `methodcaller`: 自行创建函数，创建的函数会在对象上调用参数指定的方法。还可以冻结部分参数。

In [21]:
# 使用 reduce 和 匿名函数计算阶乘
from functools import reduce
def fact(n):
    return reduce(lambda a,b: a*b, range(1, n+1))

for i in range(5):
    print(fact(i+1))

1
2
6
24
120


In [22]:
# 使用 reduce 和 operator.mul 函数计算阶乘
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1,n+1))

for i in range(5):
    print(f"{i+1}\t{fact(i+1)}")

1	1
2	2
3	6
4	24
5	120


In [23]:
# 使用 itemgetter 排序一个元组列表
metro_data = [
    ('wuhan', 'Hubei', 45678, (456, 7867)),
    ('guangzhou', 'Guangdong', 567234, (2345678, 7867)),
    ('shijiazhuang', 'Hebei', 23456, (34567, 3456)),
    ('jinan', 'Shangdong', 34567, (23456, 345)),
    ('nanjing', 'Jiangsu', 2345678, (234567, 7867)),
]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

print("===============================================================")
# 把多个参数传给 itemgetter, 它构建的函数会返回提取值构成的元组
cc_name = itemgetter(1,0)
for city in metro_data:
    print(cc_name(city))

('guangzhou', 'Guangdong', 567234, (2345678, 7867))
('shijiazhuang', 'Hebei', 23456, (34567, 3456))
('wuhan', 'Hubei', 45678, (456, 7867))
('nanjing', 'Jiangsu', 2345678, (234567, 7867))
('jinan', 'Shangdong', 34567, (23456, 345))
('Hubei', 'wuhan')
('Guangdong', 'guangzhou')
('Hebei', 'shijiazhuang')
('Shangdong', 'jinan')
('Jiangsu', 'nanjing')


In [24]:
from collections import namedtuple
from operator import attrgetter

LarLong = namedtuple('Latlong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_area = [Metropolis(name, cc, pop, LarLong(lat, long)) for name, cc, pop, (lat, long) in metro_data]
print(metro_area[0])
print(metro_area[0].coord.lat)

name_lat = attrgetter('name', 'coord.lat')
print(f"name_lat:\t{name_lat}")

for city in sorted(metro_area, key=attrgetter('coord.lat')):
    print(name_lat(city))

Metropolis(name='wuhan', cc='Hubei', pop=45678, coord=Latlong(lat=456, long=7867))
456
name_lat:	operator.attrgetter('name', 'coord.lat')
('wuhan', 456)
('jinan', 23456)
('shijiazhuang', 34567)
('nanjing', 234567)
('guangzhou', 2345678)


In [25]:
# operator 共52种
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [26]:
# methodcaller: 自行创建函数，创建的函数会在对象上调用参数指定的方法。还可以冻结部分参数。
from operator import methodcaller

s = 'the time has come'
upcase = methodcaller('upper')
print(upcase(s))

hiphenate = methodcaller('replace', ' ', '_')
print(hiphenate(s))

THE TIME HAS COME
the_time_has_come


### 10.2 functools模块

1. `partial`:
2. `partialmethod`:
3. `lru_cache`:

In [27]:
from operator import mul
from functools import partial

triple = partial(mul, 3)
print(triple(4))
print(triple(5))
print(list(map(triple, range(1,10))))

12
15
[3, 6, 9, 12, 15, 18, 21, 24, 27]


In [28]:
# 使用 partial 构建一个便利的 Unicode 规范化函数
import unicodedata, functools

nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)
print(s1==s2)
print(nfc(s1)==nfc(s2))

café café
False
True


In [31]:
# 将 partial 应用到 tag 上
from functools import partial

picture = partial(tag, 'img', cls='pic-frame')
print(picture(src='wumpus.jpeg'))
print(picture)
print(picture.func)
print(picture.args)
print(picture.keywords)

<img class='pic-frame' src='wumpus.jpeg'/>
functools.partial(<function tag at 0x00000234E34D3D08>, 'img', cls='pic-frame')
<function tag at 0x00000234E34D3D08>
('img',)
{'cls': 'pic-frame'}
